In [251]:
import time
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults
from langchain.retrievers import WikipediaRetriever
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
import nest_asyncio
nest_asyncio.apply()

def get_duckDuckGoSearch(inputs):
    query = inputs["query"]
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=1)
    search = DuckDuckGoSearchResults(api_wrapper=wrapper)
    return search.run(query)


def get_wikipediaSearch(inputs):
    query = inputs["query"]
    retriever = WikipediaRetriever(top_k_results=1)
    docs =  retriever.get_relevant_documents(query)
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed[0].page_content


def get_linkScrape(inputs):
    link = inputs["link"]
    lst = []
    lst.append(link)
    loader = AsyncChromiumLoader(lst)
    docs = loader.load()
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    return docs_transformed[0].page_content


def saveTXTfileTool(inputs):
    doc = inputs["doc"]
    file_path = f"./files/{time.strftime('%H%M%S')}.txt"
    with open(file_path, 'w') as f:
        f.write(doc)



functions_map = {
    "get_duckDuckGoSearch": get_duckDuckGoSearch,
    "get_wikipediaSearch": get_wikipediaSearch,
    "get_linkScrape": get_linkScrape,
    "saveTXTfileTool": saveTXTfileTool,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_wikipediaSearch",
            "description": "Use this tool to research.It takes a query as an argument and return content",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for.",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_duckDuckGoSearch",
            "description": "Use this tool to find the ONLY ONE link to the most relevant result for my research.It takes a query as an argument and you return ONE link.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query you will search for.",
                    },
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_linkScrape",
            "description": "You have the one website link from DuckDuckGo,Use this to get the content of the link for my research. You shoud enter the ONLY ONE link and return the content",
            "parameters": {
                "type": "object",
                "properties": {
                    "link": {
                        "type": "string",
                        "description": "The ONE link you will extract it's content. Example: 'https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/'",
                    },
                },
                "required": ["link"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "saveTXTfileTool",
            "description": "If you found the website link in DuckDuckGo and extracted the content or Wikipedia content, Use this to save the content to a .txt file for my research.You shoud enter long text that you extracted from the websites.",
            "parameters": {
                "type": "object",
                "properties": {
                    "doc": {
                        "type": "string",
                        "description": "the content extracted from the link and you will save to .txt file.",
                    },
                },
                "required": ["doc"],
            },
        },
    },
]


In [252]:
import openai as client

assistant = client.beta.assistants.create(
    name="Research Assistant",
    instructions="You help users do research in Wikipedia and DuckDuckGo. when you find the website link from DuckDuckGo and extract the content of the link and you save the content to a .txt file for my research. Don't forget to save the .txt file",
    model="gpt-4-1106-preview",
    tools=functions,
)
assistant



Assistant(id='asst_6KZ58Yd9VDXmqkQzmXLdTXcs', created_at=1713130437, description=None, file_ids=[], instructions="You help users do research in Wikipedia and DuckDuckGo. when you find the website link from DuckDuckGo and extract the content of the link and you save the content to a .txt file for my research. Don't forget to save the .txt file", metadata={}, model='gpt-4-1106-preview', name='Research Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipediaSearch', description='Use this tool to research.It takes a query as an argument and return content', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for.'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_duckDuckGoSearch', description='Use this tool to find the ONLY ONE link to the most relevant result for my research.It takes a query as an argument and you return ONE link.', para

In [253]:

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Research about the XZ backdoor",
        }
    ]
)
thread

Thread(id='thread_aMEs1bEupXXecqUyD0JVUOS4', created_at=1713130440, metadata={}, object='thread')

In [254]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
run

Run(id='run_jkufaD6OSOPNiqRZg1ITdVei', assistant_id='asst_6KZ58Yd9VDXmqkQzmXLdTXcs', cancelled_at=None, completed_at=None, created_at=1713130445, expires_at=1713131045, failed_at=None, file_ids=[], instructions="You help users do research in Wikipedia and DuckDuckGo. when you find the website link from DuckDuckGo and extract the content of the link and you save the content to a .txt file for my research. Don't forget to save the .txt file", last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_aMEs1bEupXXecqUyD0JVUOS4', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipediaSearch', description='Use this tool to research.It takes a query as an argument and return content', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for.'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition

In [255]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        args = function.arguments 
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(args)),
                "tool_call_id": action_id,
            }
        )
        print(outputs)
        
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [261]:
get_run(run.id, thread.id).status

'requires_action'

In [260]:
get_tool_outputs(run.id, thread.id)

Calling function: saveTXTfileTool with arg {"doc":"Skip to main content\n\nOpen Navigation Menu\n\nMenu\n\nStory Saved\n\nTo revisit this article, visit My Profile, then View saved stories.\n\nClose Alert\n\nThe XZ Backdoor: Everything You Need to Know\n\n  * Security\n  * Politics\n  * Gear\n  * Backchannel\n  * Business\n  * Science\n  * Culture\n  * Ideas\n  * Merch\n\nStory Saved\n\nTo revisit this article, visit My Profile, then View saved stories.\n\nClose Alert\n\nSign In\n\nSUBSCRIBE\n\n# Get WIRED\n\n# for just ~~$29.99~~ $5\n\nSUBSCRIBE\n\nSearch\n\nSearch\n\n  * Security\n  * Po"}
[{'output': None, 'tool_call_id': 'call_zDJkx3TotHpK2dxZzO4z0n30'}]


[{'output': None, 'tool_call_id': 'call_zDJkx3TotHpK2dxZzO4z0n30'}]

In [259]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_linkScrape with arg {"link":"https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/"}
[{'output': 'Skip to main content\n\nOpen Navigation Menu\n\nMenu\n\nStory Saved\n\nTo revisit this article, visit My Profile, then View saved stories.\n\nClose Alert\n\nThe XZ Backdoor: Everything You Need to Know\n\n  * Security\n  * Politics\n  * Gear\n  * Backchannel\n  * Business\n  * Science\n  * Culture\n  * Ideas\n  * Merch\n\nStory Saved\n\nTo revisit this article, visit My Profile, then View saved stories.\n\nClose Alert\n\nSign In\n\nSUBSCRIBE\n\n# Get WIRED\n\n# for just ~~$29.99~~ $5\n\nSUBSCRIBE\n\nSearch\n\nSearch\n\n  * Security\n  * Po', 'tool_call_id': 'call_Rqrn3ANm3XoC2kSL0jGquI8C'}]


Run(id='run_jkufaD6OSOPNiqRZg1ITdVei', assistant_id='asst_6KZ58Yd9VDXmqkQzmXLdTXcs', cancelled_at=None, completed_at=None, created_at=1713130445, expires_at=1713131045, failed_at=None, file_ids=[], instructions="You help users do research in Wikipedia and DuckDuckGo. when you find the website link from DuckDuckGo and extract the content of the link and you save the content to a .txt file for my research. Don't forget to save the .txt file", last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713130488, status='queued', thread_id='thread_aMEs1bEupXXecqUyD0JVUOS4', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipediaSearch', description='Use this tool to research.It takes a query as an argument and return content', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The query you will search for.'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefi

In [262]:
get_messages(thread.id)

user: Research about the XZ backdoor
